## **Data Collection**

### **Problem Statement**

It is the time of ecommerce and online shopping with companies or brands like Nike, Adidas, H&M started building their online presence to cater to customers to shop online. Shopping online for clothing and footwear was becoming more common. 

Reddit's [Frugal Male Fashion subreddit](https://www.reddit.com/r/frugalmalefashion) is where users share information about coupons, discounts and sales on a variety of brands and stores. Popular posts are upvoted and headlined at the top, but users can also filter for most recent or most relevant searches. The continuous postings can streamline viewers to the forum increasing advertisement revenue. 

In order to maintain a clean environment and better retain constant users online, only content that is relevant to the community will be allowed to be posted and irrelevant content will be removed. An example of irrelevant content is a user posting present or future acquisition of hype sneakers. We will be retrieving posts from Reddit's [Sneakers subreddit](https://www.reddit.com/r/sneakers) where users share their present or future acquisitions of hype sneakerwear.

Posts from `r/frugalmalefashion` will be Class 1 and posts from `r/sneakers` will be Class 0. We will be analyzing the documents to build a model that accurately classifies a post as from Class 1 but also outputs interpretable results for downstream data visualization and businesss applications.

### **Import Library**

In [1]:
!pip install psycopg2-binary

In [2]:
import pandas as pd
import json
import time
import regex as re
import requests
import psycopg2 as pg2
from datetime import datetime, timedelta
from psycopg2.extras import RealDictCursor, Json

In [3]:
%run ../sql_test.py

### **Configure Postgres SQL Server with Docker**

Define functions to programmatically connect to and insert data into database:
- **con_cur_to_db**: returns both a connection and a cursor object for database
- **execute_query**: executes query directly to database, without having to create a cursor and connection each time
- **insert_entry_json**: inserts data into database

In [4]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    con = pg2.connect(host=IP_ADDRESS,
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

We create table raw_posts to save our collected data into.

In [ ]:
query = '''CREATE TABLE raw_posts
(id SERIAL,
data JSONB);'''

In [ ]:
# query = '''DROP TABLE raw_posts'''

In [ ]:
execute_query(query, command=True)

### **Collect Posts from Subreddits**

We create a function using Reddit's pushshiftAPI that retrieves posts and saves them into a SQL Database:
- `subreddit`: domain of reddit
- `limit`: cap on how much data to retrieve
- `after`: begin date
- `before`: end date
- `sql_db`: SQL Database
- `time sleep`: rest latency

In [5]:
def streamComments(subreddit, sql_db, limit=1000, before=None, after=None):
    url = f'https://apiv2.pushshift.io/reddit/submission/search/'
    last_comment = before
    
    num_comments = 0
    while num_comments < limit:
            params = {
              'subreddit':subreddit,
              'sort':'desc',
              'size':1000,
              'before':last_comment-1,
              'after':after,
             }
                
            response = requests.get(url, params=params)
            posts = response.json()['data']
            
            if len(posts) == 0:
                last_comment = last_comment
            else:
                last_comment = posts[-1]['created_utc']
                insert_entry_json(data = posts, 
                          tablename = sql_db)
                timestamp = posts[-1]['created_utc']
                time.sleep(1)
                num_comments+=len(posts)

### **Frugal Male Fashion Data Collection**

We will create a loop such to send a request to the pushshiftAPI to retrieve the most recent posts from `r/frugalmalefashion` with a limit of 50,000 posts.

In [ ]:
start = time.time()

end = round(time.time())
streamComments(subreddit='frugalmalefashion',
             limit=50000,
             sql_db='raw_posts',
             before=end)
    
end = time.time()
print(str(timedelta(seconds=(end-start))))

### **Sneakers Data Collection**

We will create a loop such to send a request to the pushshiftAPI to retrieve the most recent posts from `r/sneakers` with a limit of 50,000 posts.

In [ ]:
start = time.time()

end = round(time.time())
streamComments(subreddit='sneakers',
             limit=50000,
             sql_db='raw_posts',
             before=end)
    
end = time.time()
print(str(timedelta(seconds=(end-start))))

### **Retrieve Data from Postgres SQL Database**

<img src='../image/keys_of_interest.png'>

Keys of interest:
- `title`: title text of post
- `selftext`: body text of post
- `subreddit`: reddit domain
- `created_utc`: UNIX timestamp as float

In [5]:
query = """SELECT data ->> 'title'
FROM raw_posts;
"""
title = execute_query(query, dict_cur=True)

In [6]:
title = [i['?column?'] for i in title]

In [7]:
query = """SELECT data ->> 'selftext'
FROM raw_posts;
"""
selftext = execute_query(query, dict_cur=True)

In [8]:
selftext = ['' if i['?column?'] == None else i['?column?'] for i in selftext]

In [9]:
query = """SELECT data ->> 'subreddit'
FROM raw_posts;
"""
subreddit = execute_query(query, dict_cur=True)

In [10]:
subreddit = [i['?column?'] for i in subreddit]

In [11]:
query = """SELECT data ->> 'created_utc'
FROM raw_posts;
"""
created_utc = execute_query(query, dict_cur=True)

In [12]:
year = [datetime.utcfromtimestamp(int(i['?column?'])).year for i in created_utc]

In [13]:
df = pd.DataFrame({'title':title,
                   'selftext':selftext,
                   'subreddit':subreddit,
                   'year':year})

In [14]:
df.head()

,title,selftext,subreddit,year
0,How to find the best offers and deals on lapda...,DEALS &amp; SALES,frugalmalefashion,2019
1,"Columbia Fleece, various colors and sizes, $17...",,frugalmalefashion,2019
2,"Pacsun- Vans Old Skool $26.99, sizes 9-13, fre...",https://www.pacsun.com/vans/old-skool-green-wh...,frugalmalefashion,2019
3,Massive Toad&amp;Co Sale (Some items 90% off),"Noooo idea if this was an error or not, but ju...",frugalmalefashion,2019
4,CHUP socks for $10 with code 30MORE,,frugalmalefashion,2018


#### **Remove Duplicates**

In [15]:
df.drop_duplicates(inplace=True)

#### **Missing Data**

In [16]:
df.isna().sum()

title        0
selftext     0
subreddit    0
year         0
dtype: int64

There are no missing data.

We will remove the following posts which indicate:
- **[removed]**: the post was irrelevant to the community or inappropriate and was removed by subreddit.
- **[deleted]**: the user removed own post.

In [17]:
df = df[(df['selftext'] != '[removed]') & 
              (df['title'] != '[removed]') &
              (df['selftext'] != '[deleted]')]

#### **Combined Title and Selftext**

We will combined `title` and `selftext` into `text` and drop the original columns.

In [18]:
df['text'] = df['title'] + ' ' + df['selftext']
df.drop(['title','selftext'], axis=1, inplace = True)

#### **Observe Time Range of Data**

In [19]:
df['year'].value_counts()

2018    34747
2019    22953
2015    10020
2014     6571
2017     5903
2016     4530
Name: year, dtype: int64

#### **Define Classes**

We will classify posts from `frugalmalefashion` to be `Class 1` representing relevant content and posts from `sneakers` to be `Class 0` representing irrelevant content.

In [20]:
df['subreddit'] = [1 if i=='frugalmalefashion' else 0 for i in df['subreddit']]

In [21]:
df['subreddit'].value_counts()

0    45910
1    38814
Name: subreddit, dtype: int64

In [22]:
df['subreddit'].value_counts(normalize=True)

0    0.541877
1    0.458123
Name: subreddit, dtype: float64

There is a slight imbalance of classes with `Class 0` being the majority class of 54.19% of the data.

### **Preprocessing on Text**

We will use regular expressions to remove words, symbols, characters that won't contribute real signal to our model.

#### Subreddit Name

In [23]:
df['text'] = df['text'].map(lambda x: re.sub('\s[\/]?r\/[^\s+]+', ' ', x))

#### Links

In [24]:
df['text'] = df['text'].map(lambda x: re.sub('http[s]?:\/\/[^\s]*', ' ', x))

#### Digits

In [25]:
df['text'] = df['text'].map(lambda x: re.sub('[0-9]+', '', x))

#### Lowercase

In [26]:
df['text'] = df['text'].map(lambda x: x.lower())

#### Special Characters

In [27]:
df['text'] = df['text'].map(lambda x: re.sub('[(){}[\]`~!@#$%^&*-_=+;,.<>?]+', '', x))

#### Newlines

In [28]:
df['text'] = df['text'].map(lambda x: x.replace('\n', ''))

In [29]:
df['text'] = df['text'].map(lambda x: x.replace('\r', ''))

In [30]:
df['text'] = df['text'].map(lambda x: x.replace('\t', ''))

### Strip whitespace

Strip leading and ending whitespace

In [31]:
df['text'] = df['text'].map(lambda x: x.strip())

Strip duplicate whitespace

In [32]:
df['text'] = df['text'].map(lambda x: ' '.join(x.split()))

### **Save out Dataframe as csv**

In [33]:
df.to_csv('../data/df.csv')